# IMPORT

In [1]:
from __future__ import annotations

from qiskit import QuantumCircuit
from transpiler.qasm2_reader import load_qasm, from_qasm2_to_braket
from bqskit.ext import bqskit_to_qiskit,qiskit_to_bqskit
from qiskit_aer import AerSimulator
from qiskit import transpile
import pennylane as qml
import numpy as np
from braket.devices import LocalSimulator
from braket.circuits import Circuit as braket_Circuit
from transpiler.transpile import transpile_circuit as bqskit_transpiler
from transpiler.transpile import get_noisy_counts

In [2]:
# Create a 2-qubit quantum circuit
qc = QuantumCircuit(2)

# Apply gates using QuantumCircuit methods
qc.h(0)  # Hadamard gate on qubit 0
qc.h(1)  # Hadamard gate on qubit 1
qc.cx(1, 0)  # CNOT gate (control: qubit 1, target: qubit 0)
qc.rx(1.1561, 1)  # RX rotation on qubit 1
qc.h(0)  # Hadamard gate on qubit 0
qc.rzz(1, 0, 1)  # RZZ gate with parameter 1 (qubits 0 and 1)
qc.ry(3, 1)  # RY rotation on qubit 1
qc.rxx(2.14584545, 0, 1)  # RXX gate with parameter 2.14584545 (qubits 0 and 1)

    
qc_qiskit = qc
qc_bqskit=qiskit_to_bqskit(qc)
print(qc)

# Initialize the latest AerSimulator
simulator = AerSimulator(method="statevector")

# Transpile the circuit for the simulator
qc_transpiled = transpile(qc_qiskit, simulator)

# Add instruction to save statevector
qc_transpiled.save_statevector()

# Run the circuit in statevector mode
result = simulator.run(qc_transpiled).result()

# Get the statevector
statevector = result.get_statevector()

# Print the statevector
print("Not transpiled result:",np.abs(np.array(statevector))**2)#check

     ┌───┐┌───┐    ┌───┐                      ┌──────────────┐
q_0: ┤ H ├┤ X ├────┤ H ├──────■───────────────┤0             ├
     ├───┤└─┬─┘┌───┴───┴────┐ │ZZ(1) ┌───────┐│  Rxx(2.1458) │
q_1: ┤ H ├──■──┤ Rx(1.1561) ├─■──────┤ Ry(3) ├┤1             ├
     └───┘     └────────────┘        └───────┘└──────────────┘
Not transpiled result: [0.10533642 0.41539816 0.12272557 0.35653985]


In [3]:
# Step 3: Transpile the circuit to MS gates
qc_qiskit_transpiled = bqskit_transpiler(qc_qiskit,device="Forte")
print(qc_qiskit_transpiled)

     ┌─────────────┐                              ┌──────────────┐»
q_0: ┤ Gpi(1.3558) ├──────────────────■───────────┤ Gpi2(2.7116) ├»
     ├─────────────┴┐┌──────────────┐ │ZZ(1.2249) ├──────────────┤»
q_1: ┤ Gpi2(3.1416) ├┤ Gpi2(3.7534) ├─■───────────┤ Gpi2(3.5004) ├»
     └──────────────┘└──────────────┘             └──────────────┘»
«      ┌──────────────┐ ┌──────────────┐              ┌──────────────┐»
«q_0: ─┤ Gpi2(2.6293) ├─┤ Gpi2(5.7709) ├─■────────────┤ Gpi(0.57039) ├»
«     ┌┴──────────────┴┐├──────────────┤ │ZZ(0.99575) ├─────────────┬┘»
«q_1: ┤ Gpi2(0.081795) ├┤ Gpi2(3.3431) ├─■────────────┤ Gpi(1.9247) ├─»
«     └────────────────┘└──────────────┘              └─────────────┘ »
«     ┌──────────────┐
«q_0: ┤ Gpi2(4.7124) ├
«     ├──────────────┤
«q_1: ┤ Gpi2(1.5708) ├
«     └──────────────┘


# Convert Qiskit circuit to PennyLane


In [4]:
qml_circuit = qml.from_qiskit(qc_qiskit_transpiled)
print("Transpiled result",np.abs(qml.matrix(qml_circuit, wire_order=[0, 1])().T[0])**2)#check
print("Not transpiled result:",np.abs(np.array(statevector))**2)#check

Transpiled result [0.10533642 0.12272557 0.41539817 0.35653985]
Not transpiled result: [0.10533642 0.41539816 0.12272557 0.35653985]


# NOISE

In [5]:
print("Noise 0     simulation result:", get_noisy_counts(qc_qiskit_transpiled,0.00,0.000,0.00))
print("Noisy     simulation result:", get_noisy_counts(qc_qiskit_transpiled,0.01,0.001,0.01))
print("Noiseless simulation result:",np.abs(qml.matrix(qml_circuit, wire_order=[0, 1])().T[0])**2)#check

Noise 0     simulation result: [0.10533642 0.12272557 0.41539817 0.35653985]
Noisy     simulation result: [0.11298906 0.12789764 0.40671503 0.35239828]
Noiseless simulation result: [0.10533642 0.12272557 0.41539817 0.35653985]


# Vibratium

In [6]:
bracket_qc=from_qasm2_to_braket("circuit.qasm")


In [7]:
print(bracket_qc)

T  : │        0        │      1       │      2       │     3      │      4       │      5       │      6       │     7      │      8      │      9       │      10       │ 11  │
                        ┌───────────┐                 ┌──────────┐ ┌────────────┐ ┌────────────┐ ┌────────────┐ ┌──────────┐ ┌───────────┐ ┌────────────┐                 ┌───┐ 
q0 : ───StartVerbatim───┤ GPi(1.36) ├─────────────────┤ ZZ(1.22) ├─┤ GPi2(2.71) ├─┤ GPi2(2.63) ├─┤ GPi2(5.77) ├─┤ ZZ(1.00) ├─┤ GPi(0.57) ├─┤ GPi2(4.71) ├───EndVerbatim───┤ M ├─
              ║         └───────────┘                 └────┬─────┘ └────────────┘ └────────────┘ └────────────┘ └────┬─────┘ └───────────┘ └────────────┘        ║        └───┘ 
              ║         ┌────────────┐ ┌────────────┐ ┌────┴─────┐ ┌────────────┐ ┌────────────┐ ┌────────────┐ ┌────┴─────┐ ┌───────────┐ ┌────────────┐        ║        ┌───┐ 
q1 : ─────────╨─────────┤ GPi2(3.14) ├─┤ GPi2(3.75) ├─┤ ZZ(1.22) ├─┤ GPi2(3.50) ├─┤ GPi2(0.08) ├─┤ GPi2(3.34) ├─┤ Z

In [8]:
device = LocalSimulator() # Choose the local simulator
shots = 100000  # Number of times to execute the circuit

result = device.run(bracket_qc, shots=shots).result()
counts = result.measurement_probabilities  # Get the shot results

print(counts)  # Dictionary {bitstring: counts}
print("Noiseless simulation result:",np.abs(qml.matrix(qml_circuit, wire_order=[0, 1])().T[0])**2)#check

{'11': 0.35584, '10': 0.4158, '00': 0.10526, '01': 0.1231}
Noiseless simulation result: [0.10533642 0.12272557 0.41539817 0.35653985]


# GATE braket TEST

In [9]:
print(braket_Circuit().gpi(0,1))

T  : │    0     │
      ┌────────┐ 
q0 : ─┤ GPi(1) ├─
      └────────┘ 
T  : │    0     │


In [10]:
braket_Circuit().gpi(0,0.445).to_unitary()

array([[0.        +0.j        , 0.90261067-0.43045788j],
       [0.90261067+0.43045788j, 0.        +0.j        ]])

In [11]:
phi=0.445
np.array([
            [0, np.exp(-1j * phi)],
            [np.exp(1j * phi), 0]
        ])

array([[0.        +0.j        , 0.90261067-0.43045788j],
       [0.90261067+0.43045788j, 0.        +0.j        ]])

In [12]:
braket_Circuit().gpi2(0,0.445).to_unitary()

array([[ 0.70710678+0.j        , -0.30437969-0.63824212j],
       [ 0.30437969-0.63824212j,  0.70710678+0.j        ]])

In [13]:
 # Extract the phase parameter
factor = 1 / np.sqrt(2)  # Normalization factor
exp_pos = -1j * np.exp(1j * phi)
exp_neg = -1j * np.exp(-1j * phi)

factor * np.array([
            [1, exp_neg],
            [exp_pos, 1]
        ])

array([[ 0.70710678+0.j        , -0.30437969-0.63824212j],
       [ 0.30437969-0.63824212j,  0.70710678+0.j        ]])

In [14]:
phi1=0.2455
phi2=0.4157
theta=0.11545
q1=0
q2=1
braket_Circuit().ms(q1, q2, phi1, phi2,theta).to_unitary()

array([[ 0.99833437+0.j        ,  0.        +0.j        ,
         0.        +0.j        , -0.03542718-0.0455345j ],
       [ 0.        +0.j        ,  0.99833437+0.j        ,
         0.009772  -0.05685934j,  0.        +0.j        ],
       [ 0.        +0.j        , -0.009772  -0.05685934j,
         0.99833437+0.j        ,  0.        +0.j        ],
       [ 0.03542718-0.0455345j ,  0.        +0.j        ,
         0.        +0.j        ,  0.99833437+0.j        ]])

In [15]:
phi0=phi1
phi1=phi2

cos_theta = np.cos(theta/2)
sin_theta = np.sin(theta/2)

e_pos = -1j * np.exp(1j * (phi0 + phi1))
e_neg = -1j * np.exp(-1j * (phi0 + phi1))
e_diff_pos = -1j * np.exp(1j * (phi0 - phi1))
e_diff_neg = -1j * np.exp(-1j * (phi0 - phi1))

np.array([
    [cos_theta, 0, 0, e_neg * sin_theta],
    [0, cos_theta, e_diff_neg * sin_theta, 0],
    [0, e_diff_pos * sin_theta, cos_theta, 0],
    [e_pos * sin_theta, 0, 0, cos_theta]
])

array([[ 0.99833437+0.j        ,  0.        +0.j        ,
         0.        +0.j        , -0.03542718-0.0455345j ],
       [ 0.        +0.j        ,  0.99833437+0.j        ,
         0.009772  -0.05685934j,  0.        +0.j        ],
       [ 0.        +0.j        , -0.009772  -0.05685934j,
         0.99833437+0.j        ,  0.        +0.j        ],
       [ 0.03542718-0.0455345j ,  0.        +0.j        ,
         0.        +0.j        ,  0.99833437+0.j        ]])

In [16]:
braket_Circuit().zz(0,1,np.pi).to_unitary()

array([[6.123234e-17-1.j, 0.000000e+00+0.j, 0.000000e+00+0.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 6.123234e-17+1.j, 0.000000e+00+0.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 0.000000e+00+0.j, 6.123234e-17+1.j,
        0.000000e+00+0.j],
       [0.000000e+00+0.j, 0.000000e+00+0.j, 0.000000e+00+0.j,
        6.123234e-17-1.j]])